# Исследование надежности заемщиков

**Цель проекта:**

Целью данного проекта является исследование надежности заёмщиков для кредитного отдела банка с целью выяснить, какие факторы, такие как семейное положение и количество детей клиента, влияют на вероятность возврата кредита в срок. Результаты данного исследования будут использованы при построении модели кредитного скоринга.

**Описание данных**

- *children* — количество детей в семье
- *days_employed* — общий трудовой стаж в днях
- *dob_years* — возраст клиента в годах
- *education* — уровень образования клиента
- *education_id* — идентификатор уровня образования
- *family_status* — семейное положение
- *family_status_id* — идентификатор семейного положения
- *gender* — пол клиента
- *income_type* — тип занятости
- *debt* — имел ли задолженность по возврату кредитов
- *total_income* — ежемесячный доход
- *purpose* — цель получения кредита

## Содержание работы:

1) [Изучение данных:](#1)

- Загрузка данных из предоставленного набора.
- Ознакомление с общей информацией о данных (структура, типы данных, наличие пропусков и дубликатов).

2) [Предобработка данных:](#2)

- Обработка пропущенных значений: заполнение или удаление.
- Обработка дубликатов, если они есть.
- Замена типов данных, если необходимо.

3) [Категоризация данных](#3)

4) [Анализ данных:](#4)

- Исследование статистики по различным параметрам (количество детей, семейное положение и другие).
- Анализ влияния различных факторов на возврат кредита в срок.

5) [Выводы:](#5)

- Выводы на основе анализа данных.
- Ответ на вопрос о влиянии семейного положения и количества детей на возврат кредита в срок.


## Изучение данных <a id="1"></a> 

In [1]:
import pandas as pd 

In [2]:
data = pd.read_csv('data.csv')

In [3]:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Можно сразу заметить что в столбце *days_employed* находятся отрицательные значения, обработаем их позже, а пока посмотрим общую информацию о датасете

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Видно что есть пропуски в столбцах *days_employed* и *total_income*

## Предобработка данных <a id="2"></a> 

Посчитаем пропуски

In [5]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Столбец total_income хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполним пропуски в этом столбце медианным значением по каждому типу из столбца income_type.

In [6]:
for i in data['income_type'].unique():
    data.loc[(data['income_type'] == i) & (data['total_income'].isna()), 'total_income'] \
    = data.loc[data['income_type'] == i, 'total_income'].median()
data.groupby('income_type')['total_income'].median()

income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        150447.935283
компаньон          172357.950966
пенсионер          118514.486412
предприниматель    499163.144947
сотрудник          142594.396847
студент             98201.625314
Name: total_income, dtype: float64

Применим модуль для отрицательных значений

In [7]:
#data['days_employed'] = data['days_employed'].apply(abs)
data['days_employed'] = abs(data['days_employed'])

Посмотрим на перечень уникальных значений столбца *children*

In [8]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

Видно, что есть аномальные значения (-1 и 20). Удалим строки с ними

In [9]:
data = data.loc[(data['children'] != -1) & (data['children'] != 20)]

In [10]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

Заполним пропуски в столбце *days_employed* медианными значениями по каждому типу занятости *income_type*

In [11]:
for i in data['income_type'].unique():
    data.loc[(data.loc[:, 'income_type'] == i) & (data.loc[:, 'days_employed'].isna()), 'days_employed'] \
    = data.loc[data.loc[:, 'income_type']== i, 'days_employed'].median()

In [12]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Все пропуски заполнены, приступим к смене типов данных и обработке дубликатов

In [13]:
data['total_income'] = data['total_income'].astype('int')

Обработаем неявные дубликаты в столбце *education*

In [14]:
data['education'] = data['education'].str.lower()

Проверим, сколько всего строк дубликатов в нашем датасете и удалим их

In [15]:
data.duplicated().sum()

71

In [16]:
data = data.drop_duplicates()

## Категоризация данных <a id="3"></a> 

Для того что бы поделить всех кредитополучателей по категориям создадим столбец *total_income_category* с категориями:
- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.

In [17]:
def categorize_income(total_income):
    if total_income <= 30000:
        return 'E'
    if total_income <= 50000:
        return 'D'
    if total_income <= 200000:
        return 'C'
    if total_income <= 1000000:
        return 'B'
    return 'A'

In [18]:
data['total_income_category'] = data['total_income'].apply(categorize_income)
data.total_income_category.head(5)

0    B
1    C
2    C
3    B
4    C
Name: total_income_category, dtype: object

Поcмотрим на перечень уникальных целей взятия кредита из столбца *purpose*

In [19]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Напишем функцию, которая на основании данных из столбца *purpose* сформирует новый столбец *purpose_category*, в который войдут следующие категории:

- 'операции с автомобилем',
- 'операции с недвижимостью',
- 'проведение свадьбы',
- 'получение образования'.

In [20]:
def categorize_purpose(purpose): 
    for row in purpose.split():
        if 'автомобил' in str(row):
            return 'операции с автомобилем'
        if 'жиль' in str(row) or 'недвижимост' in str(row):
            return 'операции с недвижимостью'
        if 'свадьб'in str(row):
            return 'проведение свадьбы'
        if 'образован' in str(row):
            return 'получение образования'

In [21]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы


## Анализ данных<a id="4"></a> 

Посмотрим уникальные значения в столбце 'gender':

In [22]:
data['gender'].unique()

array(['F', 'M', 'XNA'], dtype=object)

Посмотрим на пол 'XNA':

In [23]:
data.loc[data['gender'] == 'XNA']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
10701,0,2358.600502,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905,покупка недвижимости,B,операции с недвижимостью


Удалим эту строку и проверим

In [24]:
data = data.drop(labels = [10701], axis=0)
data.loc[data['gender'] == 'XNA']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category


Проверим, есть ли зависимость между количеством детей и возвратом кредита в срок?

In [25]:
group_children = data.groupby('children').agg({'debt': ['sum', 'count', 'mean']}) # группировка по кол-ву детей
group_children['debt']['mean'] = group_children['debt']['mean'] #подсчёт доли "проблемных" кредитов
group_children.columns = group_children.columns.droplevel() #удаляем верхний уровень колонок 'debt'
group_children['mean'] = group_children['mean'] * 100 # переводим в %
group_children.sort_values('mean') #сортируем по доле проблемных кредитов

,sum,count,mean
children,,,
5,0,9,0.000000
0,1063,14090,7.544358
3,27,330,8.181818
1,444,4808,9.234609
2,194,2052,9.454191
4,4,41,9.756098


Из сгруппированной таблицы видно, что процент проблемных кредитов самый большой среди клиентов с 1, 2 и 4 детьми. Лучший показатель у клиентов с 5 детьми, без детей и с 3 детьми. Для более глубокого анализа влияния кол-ва детей на просрочку можно посмотреть % проблемных кредитов в разрезе семейного статуса:

In [26]:
pivot_children = data.pivot_table(index=['family_status'], columns='children', values='debt', aggfunc=['mean'])
pivot_children * 100 

mean                                              \
children                      0          1          2          3          4   
family_status                                                                 
Не женат / не замужем  9.283820  11.581292  12.000000  12.500000  50.000000   
в разводе              7.015306   6.730769   8.641975   9.090909   0.000000   
вдовец / вдова         6.257379   9.090909  15.000000   0.000000   0.000000   
гражданский брак       8.391352  11.859296   8.746356  14.285714   0.000000   
женат / замужем        6.909480   8.268908   9.458578   6.827309  10.344828   

                            
children                 5  
family_status               
Не женат / не замужем  NaN  
в разводе              NaN  
вдовец / вдова         NaN  
гражданский брак       0.0  
женат / замужем        0.0

Так же, рассмотрим % проблемных кредитов у клиентов с разным кол-вом детей в разрезе уровня дохода:

In [27]:
pivot_children = data.pivot_table(index=['total_income_category'], columns='children', values='debt', aggfunc=['mean'])
pivot_children * 100

mean                                              \
children                       0          1          2          3          4   
total_income_category                                                          
A                       8.333333  11.111111   0.000000   0.000000        NaN   
B                       6.779661   7.540703   8.167331   4.545455  14.285714   
C                       7.831268   9.854015   9.854497   9.322034   8.823529   
D                       4.938272   6.060606  11.764706  20.000000        NaN   
E                      11.764706   0.000000   0.000000        NaN        NaN   

                            
children                 5  
total_income_category       
A                      NaN  
B                      0.0  
C                      0.0  
D                      0.0  
E                      NaN

Видно, что с увеличением детей, вне зависимости от семейного статуса и уровня дохода, показатель просрочки в среднем ухудшается.

**Вывод**: кредитополучатели без детей являются более ответственными, чем клиенты, у которых есть дети. Видимо, наличие ребенка накладывает на клиента дополнительные финансовое бремя (обязательства) и клиенты либо менее ответственно относятся к своим кредитным обязательствам, либо не имеют возможности в срок оплачивать обязательства.

Проверим, есть ли зависимость между семейным положением и возвратом кредита в срок

In [28]:
group_family_status = data.groupby('family_status').agg({'debt': ['sum', 'count','mean']}) # группировка по кол-ву детей
group_family_status.columns = group_family_status.columns.droplevel()
group_family_status['mean'] = group_family_status['mean'] * 100
group_family_status.sort_values('mean')

,sum,count,mean
family_status,,,
вдовец / вдова,63,951,6.624606
в разводе,84,1189,7.064760
женат / замужем,927,12261,7.560558
гражданский брак,385,4133,9.315267
Не женат / не замужем,273,2796,9.763948


**Вывод: самые отвественные клиенты принадлежат категории "вдовец/вдова", затем идут клиенты из категорий "в разводе", "женат/замужем"**

Посмотрим, есть ли зависимость между уровнем дохода и возвратом кредита в срок

Для лучшего восприятия обозначим группы как

- A - очень высокий
- B - высокий
- С - средний
- D - низкий
- E - очень низкий

In [29]:
def rename_income_category(category):
    if category == 'E':
        return 'очень низкий'
    elif category == 'D':
        return 'низкий'
    elif category == 'C':
        return 'средний'
    elif category == 'B':
        return 'высокий'
    elif category == 'A':
        return 'очень высокий'
data['new_income_category'] = data['total_income_category'].apply(rename_income_category)

In [30]:
group_total_income = data.groupby('new_income_category').agg({'debt': ['sum', 'count', 'mean']}) # группировка по кол-ву детей
group_total_income.columns = group_total_income.columns.droplevel()
group_total_income['mean'] = group_total_income['mean'] * 100
group_total_income.sort_values('mean')

,sum,count,mean
new_income_category,,,
низкий,21,349,6.017192
высокий,354,5013,7.061640
очень высокий,2,25,8.000000
средний,1353,15921,8.498210
очень низкий,2,22,9.090909


Данных по группам *очень низкий* и *очень высокий* мало. объединим их с группами *низкий* и *высокий* соответственно и сгруппируем заново.

In [31]:
data.loc[data['new_income_category'] == 'очень низкий', 'new_income_category'] = 'низкий'
data.loc[data['new_income_category'] == 'очень высокий', 'new_income_category'] = 'высокий'
group_total_income = data.groupby('new_income_category').agg({'debt': ['sum', 'count', 'mean']}) # группировка по кол-ву детей
group_total_income.columns = group_total_income.columns.droplevel()
group_total_income['mean'] = group_total_income['mean'] * 100
group_total_income.sort_values('mean')

,sum,count,mean
new_income_category,,,
низкий,23,371,6.199461
высокий,356,5038,7.066296
средний,1353,15921,8.498210


**Вывод: проослеживается следующая зависимость: у клиентов с низкими доходами меньший % просрочек кредитов, в то время как самый большой % просрочек у людей со средним уровнем дохода.**

Проверим, как разные цели кредита влияют на его возврат в срок

In [32]:
group_purpose = data.groupby('purpose_category').agg({'debt': ['sum', 'count','mean']}) # группировка по кол-ву детей
group_purpose.columns = group_purpose.columns.droplevel() #подсчёт доли "проблемных" кредитов
group_purpose['mean'] = group_purpose['mean'] * 100
group_purpose.sort_values('mean')

,sum,count,mean
purpose_category,,,
операции с недвижимостью,780,10750,7.255814
проведение свадьбы,183,2313,7.911803
получение образования,369,3988,9.252758
операции с автомобилем,400,4279,9.347978


**Вывод: меньше всего проблем возникает с клиентами, взявшими кредит на операции с недвижимостью и проведение свадьбы.**

Разделим клиентов на 3 возрастные группы - 0-35/36-50/50+

In [33]:
def age_division(age):
    if age <= 35:
        return 'возрастная группа 0-35'
    elif age <= 50:
        return 'возрастная группа 36-50'
    else:
        return 'возрастная группая 60+'
data['new_age_category'] = data['dob_years'].apply(age_division)

In [34]:
pivot_children = data.pivot_table(index=['education', 'new_age_category'], columns='gender',
                                  values='debt', aggfunc=['sum','count','mean'])
pivot_children['mean'] = pivot_children['mean'] * 100
#pivot_children.columns = pivot_children.columns.droplevel(1)
pivot_children #.sort_values('mean')

sum      count             mean  \
gender                                         F    M     F     M          F   
education           new_age_category                                           
высшее              возрастная группа 0-35    78   54  1363   731   5.722671   
                    возрастная группа 36-50   56   33  1318   603   4.248862   
                    возрастная группая 60+    33   24   845   367   3.905325   
начальное           возрастная группа 0-35     6    7    25    38  24.000000   
                    возрастная группа 36-50    7    6    40    52  17.500000   
                    возрастная группая 60+     3    2    98    29   3.061224   
неоконченное высшее возрастная группа 0-35    26   17   275   180   9.454545   
                    возрастная группа 36-50   13    7   120    82  10.833333   
                    возрастная группая 60+     2    3    64    19   3.125000   
среднее             возрастная группа 0-35   276  239  2243  1788  12.304949   
                    возрастная группа 36-50  291  236  3761  2094   7.737304   
                    возрастная группая 60+   198  115  3938  1251   5.027933   
ученая степень      возрастная группа 36-50    0    0     1     2   0.000000   
                    возрастная группая 60+     0    0     1     2   0.000000   

                                                        
gender                                               M  
education           new_age_category                    
высшее              возрастная группа 0-35    7.387141  
                    возрастная группа 36-50   5.472637  
                    возрастная группая 60+    6.539510  
начальное           возрастная группа 0-35   18.421053  
                    возрастная группа 36-50  11.538462  
                    возрастная группая 60+    6.896552  
неоконченное высшее возрастная группа 0-35    9.444444  
                    возрастная группа 36-50   8.536585  
                    возрастная группая 60+   15.789474  
среднее             возрастная группа 0-35   13.366890  
                    возрастная группа 36-50  11.270296  
                    возрастная группая 60+    9.192646  
ученая степень      возрастная группа 36-50   0.000000  
                    возрастная группая 60+    0.000000

In [35]:
pivot_children = data.pivot_table(index=['income_type', 'new_age_category'], columns='gender',
                                  values='debt', aggfunc=['sum','count','mean'])
pivot_children['mean'] = pivot_children['mean'] * 100
#pivot_children.columns = pivot_children.columns.droplevel(1)
pivot_children#.sort_values('mean')

sum          count          \
gender                                       F      M       F       M   
income_type     new_age_category                                        
безработный     возрастная группа 0-35     NaN    1.0     NaN     1.0   
                возрастная группа 36-50    0.0    NaN     1.0     NaN   
в декрете       возрастная группа 36-50    1.0    NaN     1.0     NaN   
госслужащий     возрастная группа 0-35    23.0   14.0   336.0   172.0   
                возрастная группа 36-50   27.0   11.0   505.0   158.0   
                возрастная группая 60+     8.0    3.0   227.0    53.0   
компаньон       возрастная группа 0-35    98.0   76.0  1145.0   789.0   
                возрастная группа 36-50   86.0   62.0  1461.0   772.0   
                возрастная группая 60+    26.0   26.0   567.0   312.0   
пенсионер       возрастная группа 0-35     1.0    1.0    27.0    10.0   
                возрастная группа 36-50   11.0    7.0   149.0    80.0   
                возрастная группая 60+   153.0   43.0  2941.0   605.0   
предприниматель возрастная группа 0-35     0.0    NaN     1.0     NaN   
                возрастная группая 60+     NaN    0.0     NaN     1.0   
сотрудник       возрастная группа 0-35   264.0  225.0  2397.0  1764.0   
                возрастная группа 36-50  242.0  202.0  3123.0  1823.0   
                возрастная группая 60+    49.0   72.0  1211.0   697.0   
студент         возрастная группа 0-35     NaN    0.0     NaN     1.0   

                                               mean              
gender                                            F           M  
income_type     new_age_category                                 
безработный     возрастная группа 0-35          NaN  100.000000  
                возрастная группа 36-50    0.000000         NaN  
в декрете       возрастная группа 36-50  100.000000         NaN  
госслужащий     возрастная группа 0-35     6.845238    8.139535  
                возрастная группа 36-50    5.346535    6.962025  
                возрастная группая 60+     3.524229    5.660377  
компаньон       возрастная группа 0-35     8.558952    9.632446  
                возрастная группа 36-50    5.886379    8.031088  
                возрастная группая 60+     4.585538    8.333333  
пенсионер       возрастная группа 0-35     3.703704   10.000000  
                возрастная группа 36-50    7.382550    8.750000  
                возрастная группая 60+     5.202312    7.107438  
предприниматель возрастная группа 0-35     0.000000         NaN  
                возрастная группая 60+          NaN    0.000000  
сотрудник       возрастная группа 0-35    11.013767   12.755102  
                возрастная группа 36-50    7.748959   11.080636  
                возрастная группая 60+     4.046243   10.329986  
студент         возрастная группа 0-35          NaN    0.000000

Можно сделать вывод что с клиентами более старших возрастных групп возникает мешьше проблем. Так же, из данных можно сделать вывод что женщины более ответственны чем мужчины.

## Выводы <a id="5"></a> 

Проведенное исследование показало влияние различных факторов на вероятность просрочки платежа по кредиту.

Были поставлены следующие вопросы:

- Есть ли зависимость между количеством детей и возвратом кредита в срок?
- Есть ли зависимость между семейным положением и возвратом кредита в срок?
- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
- Как разные цели кредита влияют на его возврат в срок?

Соответственно можно выдвинуть гипотезы:

- Количество детей влияет на возврат кредита в срок
- Семейное влияет на возврат кредита в срок
- Уровень дохода влияет на возврат кредита в срок
- Цель кредита влияет на его возврат в срок

**Количество детей влияет на возврат кредита в срок**

Для точного анализа недостаточно данных. у клиентов с 5, 3 и 4 детьми данных в датафрейме меньше 2%, что не даёт полной картины.

- у клиентов с 5 детьми вероятность просрочки: 0.00 %
- у клиентов с 0 детьми вероятность просрочки: 7.54 %
- у клиентов с 3 детьми вероятность просрочки: 8.18 %
- у клиентов с 1 детьми вероятность просрочки: 9.23 %
- у клиентов с 2 детьми вероятность просрочки: 9.45 %
- у клиентов с 4 детьми вероятность просрочки: 9.76 %

**Семейное влияет на возврат кредита в срок**

- у клиентов категории *вдовец / вдова* вероятность просрочки: 6.62 %
- у клиентов категории *в разводе*  вероятность просрочки: 7.06 %
- у клиентов категории *женат / замужем*  вероятность просрочки: 7.56 %
- у клиентов категории *гражданский брак*  вероятность просрочки: 9.32 %
- у клиентов категории *Не женат / не замужем*  вероятность просрочки: 9.76 %

Люди, которые были или имеют отношения более ответственно подходят к обязательствам и уплачивают кредиты во время.

**Уровень дохода влияет на возврат кредита в срок**

Для более точного анализа необходима большая выборка данных по клиентам с низкими (категория D), очень высокими (категория A) и очень низкими доходами (категория E)

- у клиентов с уровнем дохода *низкий* вероятность просрочки: 6.20 %
- у клиентов с уровнем дохода *высокий* вероятность просрочки: 7.07 %
- у клиентов с уровнем дохода *средний* вероятность просрочки: 8.50 %

**Цель кредита влияет на его возврат в срок**

- у клиентов с целью кредита *операции с недвижимостью* вероятность просрочки: 7.26 %
- у клиентов с целью кредита *проведение свадьбы* вероятность просрочки: 7.91 %
- у клиентов с целью кредита *получение образования* вероятность просрочки: 9.25 %
- у клиентов с целью кредита *операции с автомобилем* вероятность просрочки: 9.35 %

Безопаснее всего выдавать кредиты на *Операции с недвижимостью*, в то время как на *Операции с автомобилем* - наоборот.